# Quick low‑$z$ Hubble–Law Fit for $H_0$ from Type Ia Supernovae (Pantheon+)

This notebook implements a **quick, low‑redshift** determination of the Hubble constant using
\[
cz \approx H_0\, d \quad (z \lesssim 0.1)
\]
and supernovae Type Ia as (standardized) candles.

**What students will do**
1. Load a SN dataset with standardized peak apparent magnitudes $m_B$ and redshifts $z$.
2. Assume (or adopt) an absolute magnitude $M_B$ (the “anchor”).
3. Compute distance modulus $\mu = m_B - M_B$ and distance
$d\,[\mathrm{Mpc}] = 10^{(\mu-25)/5}$.
4. Fit $H_0$ from a regression of $cz$ on $d$.

> **Important conceptual note:** without an external calibration of $M_B$, supernovae constrain only the combination
> $M_B - 5\log_{10}H_0$. For a classroom exercise, we **assume** $M_B$ (or you can set $H_0$ and solve for $M_B$).

We use _Pantheon_, the largest, homogenized compilation of Type Ia supernova light curves used for cosmology. It contains ~1700 spectroscopically confirmed Type Ia supernovae in redshift range: $z \approx 0.001$ to $z \sim 2.3$, with uniform light-curve fitting (SALT2 framework) and careful cross-survey calibration. The apparent magnitudes are corrected for stretch (how quickly the supernova brightens and fades) and color (how red or blue the supernova appears relative to a template).

**How is $z$ determined?** Typically from **spectroscopy** (host galaxy lines preferred), then corrected to the **CMB rest frame** and with an uncertainty contribution from **peculiar velocities**, which matter at low $z$.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

c_kms = 299792.458  # speed of light in km/s


## 1) Download the Pantheon+ table

For this exercise we need **per‑SN**:
- a redshift appropriate for the Hubble diagram (often called `zHD`, `zCMB`, or similar), and
- a standardized apparent magnitude (often `mB` / `mb`).

In [2]:
GITHUB_RAW_URL = "https://raw.githubusercontent.com/PantheonPlusSH0ES/DataRelease/refs/heads/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES.dat"
OUT_FILE = "Pantheon+SH0ES.dat"

In [7]:
import requests

r = requests.get(GITHUB_RAW_URL, timeout=30)
r.raise_for_status()

with open(OUT_FILE, "wb") as f:
  f.write(r.content)

## 2) Load the dataset

We identify columns for redshift, standardized apparent magnitude, uncertainty on magnitude.

In [8]:
df = pd.read_csv(OUT_FILE, sep=r'\s+', comment="#")

In [9]:
df.columns

Index(['CID', 'IDSURVEY', 'zHD', 'zHDERR', 'zCMB', 'zCMBERR', 'zHEL',
       'zHELERR', 'm_b_corr', 'm_b_corr_err_DIAG', 'MU_SH0ES',
       'MU_SH0ES_ERR_DIAG', 'CEPH_DIST', 'IS_CALIBRATOR', 'USED_IN_SH0ES_HF',
       'c', 'cERR', 'x1', 'x1ERR', 'mB', 'mBERR', 'x0', 'x0ERR', 'COV_x1_c',
       'COV_x1_x0', 'COV_c_x0', 'RA', 'DEC', 'HOST_RA', 'HOST_DEC',
       'HOST_ANGSEP', 'VPEC', 'VPECERR', 'MWEBV', 'HOST_LOGMASS',
       'HOST_LOGMASS_ERR', 'PKMJD', 'PKMJDERR', 'NDOF', 'FITCHI2', 'FITPROB',
       'm_b_corr_err_RAW', 'm_b_corr_err_VPEC', 'biasCor_m_b',
       'biasCorErr_m_b', 'biasCor_m_b_COVSCALE', 'biasCor_m_b_COVADD'],
      dtype='object')

In [10]:
z_col, m_col, me_col = 'zHD', 'mB', 'mBERR'

## 3) Low‑$z$ selection and distance modulus

We cut out very nearby objects to reduce the impact of peculiar velocities:
- typical classroom cut: **$z > 0.02$** (or $0.023$)
- and keep to low redshift for the linear Hubble law: **$z < 0.10$** (or $0.15$)

We are going to use -19.3 as the absolute magnitude of a reference supernova.

In [11]:
# --- User-set parameters ---
z_min = 0.02
z_max = 0.10

# Choose an absolute magnitude anchor (typical standardized SN Ia value)
M_B = -19.3

In [18]:
# grab and rename columns
sub = df[[z_col, m_col, me_col]].rename(columns={z_col: "z", m_col: "mB", me_col: "mB_err"})

## 4) Fit $H_0$ from $v \approx H_0 d$

We fit a straight line **through the origin**:
$d_i = H_0^{-1} v_i$ using weighted least squares with weights $w_i = 1/\sigma_{d,i}^2$. Then plot the distance/velocity relation and overplot the fitted Hubble law.

We plot the Hubble diagram

---